# Loading data with DisjointTimeBasedCesnetDataset

### Import

In [1]:
from tqdm import tqdm
import logging

from cesnet_tszoo.utils.enums import AgreggationType, SourceType, TimeFormat, DatasetType
from cesnet_tszoo.datasets import CESNET_TimeSeries24
from cesnet_tszoo.configs import DisjointTimeBasedConfig # Disjoint dataset MUST use DisjointTimeBasedConfig

### Setting logger

In [2]:
logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s][%(name)s][%(levelname)s] - %(message)s")

### Preparing dataset

In [3]:
disjoint_dataset = CESNET_TimeSeries24.get_dataset(data_root="/some_directory/", source_type=SourceType.INSTITUTION_SUBNETS, aggregation=AgreggationType.AGG_1_HOUR, dataset_type=DatasetType.DISJOINT_TIME_BASED, display_details=True)

[2025-08-19 11:00:11,285][wrapper_dataset][INFO] - Dataset is disjoint_time_based. Use cesnet_tszoo.configs.DisjointTimeBasedConfig



Dataset details:

    AgreggationType.AGG_1_HOUR
        Time indices: range(0, 6717)
        Datetime: (datetime.datetime(2023, 10, 9, 0, 0, tzinfo=datetime.timezone.utc), datetime.datetime(2024, 7, 14, 21, 0, tzinfo=datetime.timezone.utc))

    SourceType.INSTITUTION_SUBNETS
        Time series indices: [0 1 2 3 4 ... 543 544 545 546 547], Length=548; use 'get_available_ts_indices' for full list
        Features with default values: {'n_flows': 0, 'n_packets': 0, 'n_bytes': 0, 'tcp_udp_ratio_packets': 0.5, 'tcp_udp_ratio_bytes': 0.5, 'dir_ratio_packets': 0.5, 'dir_ratio_bytes': 0.5, 'avg_duration': 0, 'avg_ttl': 0, 'sum_n_dest_asn': 0, 'avg_n_dest_asn': 0, 'std_n_dest_asn': 0, 'sum_n_dest_ports': 0, 'avg_n_dest_ports': 0, 'std_n_dest_ports': 0, 'sum_n_dest_ip': 0, 'avg_n_dest_ip': 0, 'std_n_dest_ip': 0}
        
        Additional data: ['ids_relationship', 'weekends_and_holidays']
        


### Loading data with DataLoader

- Load data using Pytorch Dataloader.
- Last batch is never dropped (unless sliding_window is used).
- Workers affect how many processes will be used for loading data for specific set.
    - Workers set to 0, means loading will be ran on main process.
    - Set workers can be overriden in `get_*_dataloader` with parameter `workers`.
- Batch size affect how many times for every time series will be in one batch (differs when sliding window is used).
- Batch consists of: (only when sliding window is not used).
    - When `time_format` is not TimeFormat.DATETIME, then batch is one Numpy array of shape `(train_ts/val_ts/test_ts, batch_size, features_to_take + used ids)`.
    - When `time_format` is TimeFormat.DATETIME, then batch is a tuple: (Numpy array of shape `(train_ts/val_ts/test_ts, batch_size, features_to_take + used ids (without time))`, Numpy array of shape `(batch_size)`)

In [4]:
config = DisjointTimeBasedConfig(train_ts=0.5, val_ts=0.2, test_ts=0.1, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-19 11:00:11,291][combined_config][INFO] - Quick validation succeeded.
[2025-08-19 11:00:11,312][combined_config][INFO] - Finalization and validation completed successfully.
[2025-08-19 11:00:11,317][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 274/274 [00:00<00:00, 1195.09it/s]
[2025-08-19 11:00:11,564][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 109/109 [00:00<00:00, 1289.33it/s]
[2025-08-19 11:00:11,660][cesnet_dataset][INFO] - Updating config for test set.
100%|██████████| 54/54 [00:00<00:00, 1089.89it/s]
[2025-08-19 11:00:11,717][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [270 183 286 398 228 ... 369 237 122 491  12], Length=274
        Val time series IDs: [489 151 487 521 412 ... 346 145 138 239 429], Length=109
        Test time series IDs: [542 390 469   3 360 ... 516 370 178 455 279], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

You can also change set batch sizes later with `update_dataset_config_and_initialize` or `set_batch_sizes`.

In [5]:
disjoint_dataset.update_dataset_config_and_initialize(train_batch_size=33, val_batch_size=65, test_batch_size="config")
# Or
disjoint_dataset.set_batch_sizes(train_batch_size=33, val_batch_size=65, test_batch_size="config")

[2025-08-19 11:00:11,722][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-19 11:00:11,722][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-19 11:00:11,723][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-19 11:00:11,723][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-19 11:00:11,724][cesnet_dataset][INFO] - Batch sizes has been changed successfuly.


You can also change set workers later with `update_dataset_config_and_initialize` or `set_workers`.

In [6]:
disjoint_dataset.update_dataset_config_and_initialize(train_workers=0, val_workers=0, test_workers=0, init_workers=0)
# Or
disjoint_dataset.set_workers(train_workers=0, val_workers=0, test_workers=0, init_workers=0)

[2025-08-19 11:00:11,729][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-19 11:00:11,730][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-19 11:00:11,730][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-19 11:00:11,730][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-19 11:00:11,732][cesnet_dataset][INFO] - Workers has been changed successfuly.


#### Train set

- Affected by `train_batch_size`.
- Affected by `train_workers`.

In [7]:
dataloader = disjoint_dataset.get_train_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-08-19 11:00:11,739][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 105/105 [00:02<00:00, 44.99it/s]


(274, 32, 20)

#### Val set

- Affected by `val_batch_size`.
- Affected by `val_workers`.

In [8]:
dataloader = disjoint_dataset.get_val_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-08-19 11:00:14,084][cesnet_dataset][INFO] - Created new cached val_dataloader.
100%|██████████| 32/32 [00:00<00:00, 67.60it/s]


(109, 64, 20)

#### Test set

- Affected by `test_batch_size`.
- Affected by `test_workers`.

In [9]:
dataloader = disjoint_dataset.get_test_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-08-19 11:00:14,572][cesnet_dataset][INFO] - Created new cached test_dataloader.
100%|██████████| 11/11 [00:00<00:00, 98.60it/s]


(54, 128, 20)

#### Using time_format=TimeFormat.DATETIME

In [10]:
config = DisjointTimeBasedConfig(train_ts=0.5, val_ts=0.2, test_ts=0.1, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.DATETIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-19 11:00:14,692][combined_config][INFO] - Quick validation succeeded.
[2025-08-19 11:00:14,716][combined_config][INFO] - Finalization and validation completed successfully.
[2025-08-19 11:00:14,720][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 274/274 [00:00<00:00, 2512.03it/s]
[2025-08-19 11:00:14,834][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 109/109 [00:00<00:00, 3273.47it/s]
[2025-08-19 11:00:14,872][cesnet_dataset][INFO] - Updating config for test set.
100%|██████████| 54/54 [00:00<00:00, 3175.54it/s]
[2025-08-19 11:00:14,890][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [429 118 234  63 183 ... 313 444  91 324 186], Length=274
        Val time series IDs: [280  90 422 300 105 ... 197 391 114 383 521], Length=109
        Test time series IDs: [ 47 410 256 243 176 ... 394 108 307  62  23], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

In [11]:
dataloader = disjoint_dataset.get_train_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0][0].shape) # data without time
display(batches[0][1].shape) # time

[2025-08-19 11:00:14,898][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 105/105 [00:02<00:00, 51.87it/s]


(274, 32, 19)

(32,)

#### Specifying which time series to load

- Every `get_*_dataloader` has parameter `ts_id`.
    - When `ts_id` is None, then it returns as previous examples.
    - When `ts_id` is not None, then it returns only one time series of that specified id.

In [12]:
config = DisjointTimeBasedConfig(train_ts=[177, 176, 319, 267], val_ts=None, test_ts=None, train_time_period=0.5, features_to_take="all", time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-19 11:00:16,930][combined_config][INFO] - Quick validation succeeded.
[2025-08-19 11:00:16,942][combined_config][INFO] - Finalization and validation completed successfully.
[2025-08-19 11:00:16,945][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 4/4 [00:00<00:00, 1999.43it/s]
[2025-08-19 11:00:16,949][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [177 176 319 267], Length=4
        Val time series IDs: None
        Test time series IDs: None
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: None
        Test time periods: None
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.5 0.5 0.  0. ]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Sliding window
      

In [13]:
dataloader = disjoint_dataset.get_train_dataloader(ts_id=177 , workers="config",)

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-08-19 11:00:16,958][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 105/105 [00:00<00:00, 2019.39it/s]


(1, 32, 20)

#### Sliding window

- Both `sliding_window_size` and `sliding_window_prediction_size` must be set if you want to use sliding window.
- Batch sizes are used for background caching.
- Batch consists of:
    - When `time_format` is not TimeFormat.DATETIME, then batch is a tuple: <br>
    ( <br>
        Numpy array of shape `(train_ts/val_ts/test_ts, sliding_window_size, features_to_take + used ids)`, <br>
        Numpy array of shape `(train_ts/val_ts/test_ts, sliding_window_prediction_size, features_to_take + used ids)` <br>
    ).
    - When `time_format` is TimeFormat.DATETIME, then batch is a tuple: <br>
    ( <br>
        Numpy array of shape `(train_ts/val_ts/test_ts, sliding_window_size, features_to_take + used ids (without time))`, <br>
        Numpy array of shape `(train_ts/val_ts/test_ts, sliding_window_prediction_size, features_to_take + used ids (without time))`, <br>
        Numpy array of shape `(sliding_window_size)` of time, <br>
        Numpy array of shape `(sliding_window_prediction_size)` of time, <br>
    ).
- You can modify sliding window step size with `sliding_window_step`
- You can use `set_shared_size` to set how many times time periods should share.
    - `val_time_period` takes from `train_time_period`
    - `test_time_period` takes from `val_time_period` or `train_time_period`

In [14]:
config = DisjointTimeBasedConfig(train_ts=0.5, val_ts=0.2, test_ts=0.1, train_time_period=range(0, 1000), val_time_period=range(1000, 1500), test_time_period=range(1500, 2000), features_to_take=["n_flows"], time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128,
                         sliding_window_size=22, sliding_window_prediction_size=2, sliding_window_step=2, set_shared_size=0.05)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-19 11:00:17,022][combined_config][INFO] - Quick validation succeeded.
[2025-08-19 11:00:17,042][combined_config][INFO] - Finalization and validation completed successfully.
[2025-08-19 11:00:17,046][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 274/274 [00:00<00:00, 7188.05it/s]
[2025-08-19 11:00:17,092][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 109/109 [00:00<00:00, 7024.02it/s]
[2025-08-19 11:00:17,114][cesnet_dataset][INFO] - Updating config for test set.
100%|██████████| 54/54 [00:00<00:00, 6553.79it/s]
[2025-08-19 11:00:17,123][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [420 499 240 530 439 ...  22 307 532 431 191], Length=274
        Val time series IDs: [437  89 189 146 344 ... 198 187 330 280  40], Length=109
        Test time series IDs: [247 176 210 502 251 ... 117  54 483 457 450], Length=54
    Time periods
        Train time periods: range(0, 1000)
        Val time periods: range(665, 1500)
        Test time periods: range(1165, 2000)
    Features
        Taken features: ['n_flows']
        Default values: [0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Sliding window
        Sliding window size: 22
        Sliding window prediction size: 2
        Sliding window step size: 2
    Fillers
        Filler type: None
    Transformers
        Transformer type: None
    Batch sizes
       

In [15]:
dataloader = disjoint_dataset.get_train_dataloader(workers="config")

batches = []

for sliding_window, sliding_window_prediction in tqdm(dataloader):
    batches.append((sliding_window, sliding_window_prediction))

[2025-08-19 11:00:17,132][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 489/489 [00:00<00:00, 1204.84it/s]


You can also change sliding window parameters later with `update_dataset_config_and_initialize` or `set_sliding_window`.

In [16]:
disjoint_dataset.update_dataset_config_and_initialize(sliding_window_size=22, sliding_window_prediction_size=3, sliding_window_step="config", set_shared_size="config", workers=0)
# Or
disjoint_dataset.set_sliding_window(sliding_window_size=22, sliding_window_prediction_size=3, sliding_window_step="config", set_shared_size="config", workers=0)

[2025-08-19 11:00:17,544][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-19 11:00:17,544][combined_config][INFO] - all_batch_size adjusted to 25 as it should be greater than or equal to sliding_window_size + sliding_window_prediction_size.
[2025-08-19 11:00:17,544][cesnet_dataset][INFO] - Destroyed cached train_dataloader.
[2025-08-19 11:00:17,545][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-19 11:00:17,545][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-19 11:00:17,546][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-19 11:00:17,546][cesnet_dataset][INFO] - Sliding window values has been changed successfuly.


##### Using time_format=TimeFormat.DATETIME

In [17]:
config = DisjointTimeBasedConfig(train_ts=0.5, val_ts=0.2, test_ts=0.1, train_time_period=range(0, 1000), val_time_period=range(978, 1500), test_time_period=range(1478, 2000), features_to_take=["n_flows"], time_format=TimeFormat.DATETIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128,
                         sliding_window_size=22, sliding_window_prediction_size=2, sliding_window_step=2, set_shared_size=100)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-19 11:00:17,550][combined_config][INFO] - Quick validation succeeded.
[2025-08-19 11:00:17,570][combined_config][INFO] - Finalization and validation completed successfully.
[2025-08-19 11:00:17,574][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 274/274 [00:00<00:00, 8516.23it/s]
[2025-08-19 11:00:17,610][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 109/109 [00:00<00:00, 7084.97it/s]
[2025-08-19 11:00:17,631][cesnet_dataset][INFO] - Updating config for test set.
100%|██████████| 54/54 [00:00<00:00, 7068.17it/s]
[2025-08-19 11:00:17,641][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [ 58 190 493 394 179 ... 366 426 246 257 457], Length=274
        Val time series IDs: [546 304 279 513   8 ... 508  11 110  51  24], Length=109
        Test time series IDs: [  1  27 494 533 497 ...  87  99 355 181 243], Length=54
    Time periods
        Train time periods: range(0, 1000)
        Val time periods: range(900, 1500)
        Test time periods: range(1400, 2000)
    Features
        Taken features: ['n_flows']
        Default values: [0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.DATETIME
    Sliding window
        Sliding window size: 22
        Sliding window prediction size: 2
        Sliding window step size: 2
    Fillers
        Filler type: None
    Transformers
        Transformer type: None
    Batch sizes
      

In [18]:
dataloader = disjoint_dataset.get_train_dataloader(workers="config")

batches = []

for sliding_window, sliding_window_prediction, sliding_window_times, sliding_window_prediction_times in tqdm(dataloader):
    batches.append((sliding_window, sliding_window_prediction, sliding_window_times, sliding_window_prediction_times))

[2025-08-19 11:00:17,648][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 489/489 [00:00<00:00, 1272.57it/s]


### Loading data as Dataframe

- Batch size has no effect.
- Sliding window has no effect.
- Returns every time series in `train_ts/val_ts/test_ts` with sets specified time period.
- Data is returned as Pandas Dataframe.
- Workers affect how many processes will be used for loading data for specific set.
    - Workers set to 0, means loading will be ran on main process.
    - Set workers can be overriden in `get_*_df` with parameter `workers`.

In [19]:
config = DisjointTimeBasedConfig(train_ts=0.5, val_ts=0.2, test_ts=0.1, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-19 11:00:18,039][combined_config][INFO] - Quick validation succeeded.
[2025-08-19 11:00:18,059][combined_config][INFO] - Finalization and validation completed successfully.
[2025-08-19 11:00:18,064][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 274/274 [00:00<00:00, 2788.45it/s]
[2025-08-19 11:00:18,167][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 109/109 [00:00<00:00, 3250.13it/s]
[2025-08-19 11:00:18,205][cesnet_dataset][INFO] - Updating config for test set.
100%|██████████| 54/54 [00:00<00:00, 3722.57it/s]
[2025-08-19 11:00:18,220][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [529 336 206   3 270 ... 197 211 348 424 305], Length=274
        Val time series IDs: [ 90 414 391  50 174 ... 300  68 397  93 514], Length=109
        Test time series IDs: [501 179 450 178 185 ... 407 356 317 522 488], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

#### Train set

- Affected by `train_workers`.

In [20]:
df = disjoint_dataset.get_train_df(as_single_dataframe=True, workers="config")
dfs = disjoint_dataset.get_train_df(as_single_dataframe=False, workers="config")

df.head(10)

,id_institution_subnet,id_time,n_flows,n_packets,n_bytes,sum_n_dest_asn,avg_n_dest_asn,std_n_dest_asn,sum_n_dest_ports,avg_n_dest_ports,std_n_dest_ports,sum_n_dest_ip,avg_n_dest_ip,std_n_dest_ip,tcp_udp_ratio_packets,tcp_udp_ratio_bytes,dir_ratio_packets,dir_ratio_bytes,avg_duration,avg_ttl
0,529.0,0.0,148.0,212.0,23059.0,42.0,5.25,1.16,46.0,5.75,1.49,122.0,15.250000,6.690000,0.439941,0.500000,0.320068,0.389893,4.620000,119.239998
1,529.0,1.0,151.0,186.0,17373.0,29.0,3.62,1.60,31.0,3.88,1.89,125.0,15.620000,7.110000,0.270020,0.290039,0.419922,0.489990,1.530000,117.480003
2,529.0,2.0,444.0,38223.0,36331239.0,105.0,15.00,11.90,48.0,6.86,2.34,344.0,49.139999,40.049999,0.720215,0.709961,0.330078,0.239990,7.150000,105.070000
3,529.0,3.0,417.0,28924.0,19820852.0,114.0,12.67,9.12,49.0,5.44,1.51,321.0,35.669998,27.340000,0.810059,0.830078,0.389893,0.350098,15.960000,124.339996
4,529.0,4.0,447.0,16088.0,11767370.0,83.0,10.38,6.93,36.0,4.50,1.69,350.0,43.750000,35.810001,0.819824,0.839844,0.429932,0.370117,22.840000,123.980003
5,529.0,5.0,378.0,24146.0,12827551.0,106.0,17.67,6.35,30.0,5.00,0.63,323.0,53.830002,15.130000,0.990234,1.000000,0.509766,0.459961,32.290001,96.120003
6,529.0,6.0,378.0,13621.0,6202851.0,104.0,13.00,8.19,37.0,4.62,1.06,323.0,40.380001,24.020000,0.839844,0.859863,0.399902,0.379883,26.730000,127.629997
7,529.0,7.0,258.0,7259.0,5360759.0,61.0,6.10,2.42,46.0,4.60,1.58,220.0,22.000000,13.580000,0.790039,0.819824,0.330078,0.260010,23.139999,155.860001
8,529.0,8.0,281.0,10078.0,8863791.0,65.0,8.12,6.83,33.0,4.12,1.36,228.0,28.500000,18.850000,0.779785,0.779785,0.429932,0.350098,33.480000,120.330002
9,529.0,9.0,360.0,19427.0,8395408.0,98.0,10.89,8.70,42.0,4.67,2.40,314.0,34.889999,37.139999,0.759766,0.759766,0.509766,0.469971,20.860001,113.099998


In [21]:
dfs

[      id_institution_subnet  id_time  n_flows  n_packets     n_bytes  \
 0                     529.0      0.0    148.0      212.0     23059.0   
 1                     529.0      1.0    151.0      186.0     17373.0   
 2                     529.0      2.0    444.0    38223.0  36331239.0   
 3                     529.0      3.0    417.0    28924.0  19820852.0   
 4                     529.0      4.0    447.0    16088.0  11767370.0   
 ...                     ...      ...      ...        ...         ...   
 3354                  529.0   3354.0     12.0       17.0       616.0   
 3355                  529.0   3355.0      0.0        0.0         0.0   
 3356                  529.0   3356.0     10.0       13.0       500.0   
 3357                  529.0   3357.0     29.0       54.0      2012.0   
 3358                  529.0   3358.0      0.0        0.0         0.0   
 
       sum_n_dest_asn  avg_n_dest_asn  std_n_dest_asn  sum_n_dest_ports  \
 0               42.0            5.25          

#### Val set

- Affected by `val_workers`.

In [22]:
df = disjoint_dataset.get_val_df(as_single_dataframe=True, workers="config")
dfs = disjoint_dataset.get_val_df(as_single_dataframe=False, workers="config")

df.head(10)

,id_institution_subnet,id_time,n_flows,n_packets,n_bytes,sum_n_dest_asn,avg_n_dest_asn,std_n_dest_asn,sum_n_dest_ports,avg_n_dest_ports,std_n_dest_ports,sum_n_dest_ip,avg_n_dest_ip,std_n_dest_ip,tcp_udp_ratio_packets,tcp_udp_ratio_bytes,dir_ratio_packets,dir_ratio_bytes,avg_duration,avg_ttl
0,90.0,3359.0,8.0,35.0,9042.0,8.0,2.67,1.53,8.0,2.67,1.53,8.0,2.67,1.53,0.890137,0.859863,0.320068,0.260010,0.390000,137.610001
1,90.0,3360.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.500000,0.500000,0.500000,0.500000,0.000000,0.000000
2,90.0,3361.0,6.0,7.0,319.0,6.0,3.00,1.41,6.0,3.00,1.41,6.0,3.00,1.41,0.879883,0.819824,0.290039,0.260010,5.540000,114.379997
3,90.0,3362.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.500000,0.500000,0.500000,0.500000,0.000000,0.000000
4,90.0,3363.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.500000,0.500000,0.500000,0.500000,0.000000,0.000000
5,90.0,3364.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.500000,0.500000,0.500000,0.500000,0.000000,0.000000
6,90.0,3365.0,11.0,150.0,39600.0,4.0,2.00,0.00,3.0,1.50,0.71,8.0,4.00,1.41,0.870117,0.680176,0.589844,0.320068,66.029999,94.879997
7,90.0,3366.0,12.0,213.0,35318.0,11.0,1.83,1.17,10.0,1.67,1.21,12.0,2.00,1.26,1.000000,1.000000,0.640137,0.580078,75.870003,92.879997
8,90.0,3367.0,52.0,35520.0,42003373.0,37.0,3.70,2.50,31.0,3.10,1.66,48.0,4.80,3.65,0.779785,0.759766,0.640137,0.600098,3.770000,101.639999
9,90.0,3368.0,134.0,33512.0,5629183.0,62.0,5.64,2.58,46.0,4.18,1.83,87.0,7.91,3.67,0.640137,0.569824,0.449951,0.419922,21.160000,135.419998


In [23]:
dfs

[      id_institution_subnet  id_time  n_flows  n_packets     n_bytes  \
 0                      90.0   3359.0      8.0       35.0      9042.0   
 1                      90.0   3360.0      0.0        0.0         0.0   
 2                      90.0   3361.0      6.0        7.0       319.0   
 3                      90.0   3362.0      0.0        0.0         0.0   
 4                      90.0   3363.0      0.0        0.0         0.0   
 ...                     ...      ...      ...        ...         ...   
 2010                   90.0   5369.0    144.0     4060.0   2684561.0   
 2011                   90.0   5370.0    171.0    13099.0  10857365.0   
 2012                   90.0   5371.0    141.0     3475.0   1308319.0   
 2013                   90.0   5372.0    124.0     2078.0   1173167.0   
 2014                   90.0   5373.0      0.0        0.0         0.0   
 
       sum_n_dest_asn  avg_n_dest_asn  std_n_dest_asn  sum_n_dest_ports  \
 0                8.0            2.67          

#### Test set

- Affected by `test_workers`.

In [24]:
df = disjoint_dataset.get_test_df(as_single_dataframe=True, workers="config")
dfs = disjoint_dataset.get_test_df(as_single_dataframe=False, workers="config")

df.head(10)

,id_institution_subnet,id_time,n_flows,n_packets,n_bytes,sum_n_dest_asn,avg_n_dest_asn,std_n_dest_asn,sum_n_dest_ports,avg_n_dest_ports,std_n_dest_ports,sum_n_dest_ip,avg_n_dest_ip,std_n_dest_ip,tcp_udp_ratio_packets,tcp_udp_ratio_bytes,dir_ratio_packets,dir_ratio_bytes,avg_duration,avg_ttl
0,501.0,5374.0,2329.0,34259.0,29708607.0,395.0,21.940001,8.730000,197.0,10.94,6.66,1580.0,87.779999,46.400002,0.629883,0.660156,0.529785,0.640137,4.42,129.089996
1,501.0,5375.0,2375.0,36506.0,32217128.0,403.0,22.389999,8.940000,200.0,11.11,6.76,1615.0,89.720001,44.660000,0.640137,0.660156,0.560059,0.640137,5.12,126.019997
2,501.0,5376.0,2371.0,42051.0,40556311.0,413.0,22.940001,10.030000,218.0,12.11,6.98,1584.0,88.000000,43.310001,0.640137,0.669922,0.509766,0.609863,4.63,127.650002
3,501.0,5377.0,2302.0,33299.0,29405081.0,406.0,22.559999,7.450000,195.0,10.83,7.10,1536.0,85.330002,39.099998,0.629883,0.660156,0.540039,0.629883,5.86,128.289993
4,501.0,5378.0,2744.0,87884.0,97685155.0,462.0,25.670000,15.700000,214.0,11.89,6.70,1822.0,101.220001,82.820000,0.649902,0.669922,0.500000,0.569824,3.72,128.729996
5,501.0,5379.0,5654.0,277212.0,291244687.0,567.0,31.500000,25.670000,232.0,12.89,9.56,3124.0,173.559998,214.259995,0.660156,0.669922,0.429932,0.509766,4.40,126.370003
6,501.0,5380.0,12449.0,780057.0,801842989.0,750.0,41.669998,37.450001,241.0,13.39,10.83,5376.0,298.670013,370.260010,0.660156,0.669922,0.439941,0.479980,6.63,118.110001
7,501.0,5381.0,10717.0,628113.0,613616901.0,678.0,37.669998,35.939999,240.0,13.33,10.20,5027.0,279.279999,348.489990,0.660156,0.669922,0.459961,0.500000,7.73,127.019997
8,501.0,5382.0,10361.0,679209.0,742029850.0,698.0,38.779999,36.880001,258.0,14.33,10.45,4991.0,277.279999,346.730011,0.660156,0.669922,0.459961,0.500000,5.88,127.709999
9,501.0,5383.0,9397.0,359437.0,318048508.0,721.0,40.060001,36.380001,235.0,13.06,8.77,5049.0,280.500000,341.359985,0.660156,0.669922,0.479980,0.500000,7.21,121.000000


In [25]:
dfs

[      id_institution_subnet  id_time  n_flows  n_packets     n_bytes  \
 0                     501.0   5374.0   2329.0    34259.0  29708607.0   
 1                     501.0   5375.0   2375.0    36506.0  32217128.0   
 2                     501.0   5376.0   2371.0    42051.0  40556311.0   
 3                     501.0   5377.0   2302.0    33299.0  29405081.0   
 4                     501.0   5378.0   2744.0    87884.0  97685155.0   
 ...                     ...      ...      ...        ...         ...   
 1338                  501.0   6712.0   1396.0    14807.0  12114061.0   
 1339                  501.0   6713.0   1346.0    20690.0  19462680.0   
 1340                  501.0   6714.0   1442.0    19620.0  18230337.0   
 1341                  501.0   6715.0   1357.0    14493.0  12199454.0   
 1342                  501.0   6716.0   1402.0    14950.0  13203097.0   
 
       sum_n_dest_asn  avg_n_dest_asn  std_n_dest_asn  sum_n_dest_ports  \
 0              395.0       21.940001          

### Loading data as singular Numpy array 

- Batch size has no effect.
- Sliding window has no effect.
- Returns every time series in `train_ts/val_ts/test_ts` with sets specified time period.
- Data is returned as one Numpy array.
- Follows similar rules to Dataloader batches, regarding shape (excluding sliding window parameters).
- Workers affect how many processes will be used for loading data for specific set.
    - Workers set to 0, means loading will be ran on main process.
    - Set workers can be overriden in `get_*_numpy` with parameter `workers`.

In [26]:
config = DisjointTimeBasedConfig(train_ts=0.5, val_ts=0.2, test_ts=0.1, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-19 11:00:20,734][combined_config][INFO] - Quick validation succeeded.
[2025-08-19 11:00:20,753][combined_config][INFO] - Finalization and validation completed successfully.
[2025-08-19 11:00:20,758][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 274/274 [00:00<00:00, 2624.01it/s]
[2025-08-19 11:00:20,867][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 109/109 [00:00<00:00, 3347.75it/s]
[2025-08-19 11:00:20,905][cesnet_dataset][INFO] - Updating config for test set.
100%|██████████| 54/54 [00:00<00:00, 3720.13it/s]
[2025-08-19 11:00:20,920][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [281 254 265 401  62 ... 173 445  99 329 161], Length=274
        Val time series IDs: [489  70 370 285 111 ... 217 383 512  29 205], Length=109
        Test time series IDs: [106  46 266 312 141 ... 195  60 117 114 447], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

#### Train set

- Affected by `train_workers`.

In [27]:
numpy_array = disjoint_dataset.get_train_numpy(workers="config")

display(numpy_array.shape)

(274, 3359, 20)

#### Val set

- Affected by `val_workers`.

In [28]:
numpy_array = disjoint_dataset.get_val_numpy(workers="config")

display(numpy_array.shape)

(109, 2015, 20)

#### Test set

- Affected by `test_workers`.

In [29]:
numpy_array = disjoint_dataset.get_test_numpy(workers="config")

display(numpy_array.shape)

(54, 1343, 20)

#### Using time_format=TimeFormat.DATETIME

In [30]:
config = DisjointTimeBasedConfig(train_ts=0.5, val_ts=0.2, test_ts=0.1, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.DATETIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-19 11:00:21,340][combined_config][INFO] - Quick validation succeeded.
[2025-08-19 11:00:21,366][combined_config][INFO] - Finalization and validation completed successfully.
[2025-08-19 11:00:21,369][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 274/274 [00:00<00:00, 2630.58it/s]
[2025-08-19 11:00:21,478][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 109/109 [00:00<00:00, 3038.34it/s]
[2025-08-19 11:00:21,518][cesnet_dataset][INFO] - Updating config for test set.
100%|██████████| 54/54 [00:00<00:00, 3597.29it/s]
[2025-08-19 11:00:21,536][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [337 297 149 437 516 ... 330 501 517 190 133], Length=274
        Val time series IDs: [524 539 303 286 107 ... 186  39 441 322 237], Length=109
        Test time series IDs: [382 245 403 194 467 ... 430 332 281 528 159], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

In [31]:
numpy_array, times = disjoint_dataset.get_train_numpy(workers="config")

display(numpy_array.shape)
display(times)

(274, 3359, 19)

array([datetime.datetime(2023, 10, 9, 0, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2023, 10, 9, 1, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2023, 10, 9, 2, 0, tzinfo=datetime.timezone.utc),
       ...,
       datetime.datetime(2024, 2, 25, 20, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2024, 2, 25, 21, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2024, 2, 25, 22, 0, tzinfo=datetime.timezone.utc)],
      shape=(3359,), dtype=object)